# ChatBot

In [1]:
import os
import neo4j
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
driver = neo4j.GraphDatabase.driver(
    os.environ.get("NEO4J_URL"),
    auth=(os.environ.get("NEO4J_USER"), os.environ.get("NEO4J_PASSWORD")),
)

In [4]:
from neo4j_graphrag.llm import OpenAILLM
from neo4j_graphrag.embeddings import OpenAIEmbeddings

In [5]:
llm = OpenAILLM(model_name="gpt-4o")
embedder = OpenAIEmbeddings(model="text-embedding-ada-002")

In [6]:
from neo4j_graphrag.indexes import create_vector_index

In [7]:
VECTOR_INDEX_NAME = "chunk-embeddings"

In [9]:
create_vector_index(
    driver,
    name=VECTOR_INDEX_NAME,
    label="Chunk",
    embedding_property="embedding",
    dimensions=1536,
    similarity_fn="cosine",
    fail_if_exists=False,
)
# MATCH (c:Chunk) RETURN size(c.embedding) LIMIT 1

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x785669806bd0>, keys=[])

In [8]:
from neo4j_graphrag.retrievers import VectorRetriever, VectorCypherRetriever
from neo4j_graphrag.generation import GraphRAG, RagTemplate

In [9]:
retriever = VectorRetriever(
    driver,
    index_name=VECTOR_INDEX_NAME,
    embedder=embedder,
)

In [14]:
retriever.search(query_text="where is alice living?", top_k=1)

RetrieverResult(items=[RetrieverResultItem(content="{'id': 'a3d6db06-c35d-4c87-835e-51dbf0737637', 'index': 0, 'text': 'Alice, a talented software engineer, resides in Sweden and works for Neo4j, a Swedish and US-\\nbased company renowned for its innovative graph database technology. She has been part of the \\nteam for several years, contributing to various projects and strengthening her expertise in data \\nmanagement and graph theory. Alice takes pride in her role at Neo4j, where collaboration and \\ncutting-edge proble', 'embedding': None}", metadata={'score': 0.9147700071334839, 'nodeLabels': ['Chunk', '__KGBuilder__'], 'id': '4:a7411f47-9112-4baf-995a-de5abe0a4a7e:40'})], metadata={'__retriever': 'VectorRetriever'})

In [15]:
graph_rag = GraphRAG(
    llm=llm,
    retriever=retriever,
    # prompt_template=RagTemplate(system_instructions="..")
)

In [18]:
result = graph_rag.search(query_text="where is alice living?", retriever_config={"top_k": 1})
result.answer

'Alice is living in Sweden.'

In [19]:
result = graph_rag.search(query_text="who are alice's co-workers?", retriever_config={"top_k": 1})
result.answer

"The context does not provide specific names or details about Alice's co-workers at Neo4j. It only mentions that she works at Neo4j, a company renowned for its graph database technology."

In [21]:
# result = graph_rag.search(query_text="who are alice's co-workers?", retriever_config={"top_k": 2})
# result.answer

In [9]:
retrieval_query = """
WITH node as chunk
MATCH (chunk)<-[:FROM_CHUNK]-(e:__Entity__)
OPTIONAL MATCH (e)-[r]->(middle:__Entity__)<-[r2]-(other:__Entity__) WHERE e <> other
RETURN e, type(r), middle, type(r2), other
"""

retriever = VectorCypherRetriever(
    driver,
    index_name=VECTOR_INDEX_NAME,
    embedder=embedder,
    retrieval_query=retrieval_query,
)

In [32]:
retriever.search(query_text="where is alice living?", top_k=1)

RetrieverResult(items=[RetrieverResultItem(content="<Record e=<Node element_id='4:a7411f47-9112-4baf-995a-de5abe0a4a7e:45' labels=frozenset({'__Entity__', '__KGBuilder__', 'Company'}) properties={'chunk_index': 0, 'name': 'Neo4j', 'id': 'a3d6db06-c35d-4c87-835e-51dbf0737637:2'}> type(r)='LOCATED_IN' middle=<Node element_id='4:a7411f47-9112-4baf-995a-de5abe0a4a7e:44' labels=frozenset({'__Entity__', '__KGBuilder__', 'Country'}) properties={'chunk_index': 0, 'name': 'Sweden', 'id': 'a3d6db06-c35d-4c87-835e-51dbf0737637:1'}> type(r2)='LOCATED_IN' other=<Node element_id='4:a7411f47-9112-4baf-995a-de5abe0a4a7e:43' labels=frozenset({'__Entity__', '__KGBuilder__', 'Person'}) properties={'chunk_index': 0, 'name': 'Alice', 'id': 'a3d6db06-c35d-4c87-835e-51dbf0737637:0'}>>", metadata=None), RetrieverResultItem(content="<Record e=<Node element_id='4:a7411f47-9112-4baf-995a-de5abe0a4a7e:43' labels=frozenset({'__Entity__', '__KGBuilder__', 'Person'}) properties={'chunk_index': 0, 'name': 'Alice', 'i

In [10]:
graph_rag = GraphRAG(
    llm=llm,
    retriever=retriever,
)
result = graph_rag.search(query_text="who are alice's co-workers?", retriever_config={"top_k": 1})
result.answer

"Alice's co-worker is Bob, as both work for the company Neo4j."

In [16]:
history = [
    {
        "role": "user",
        "content": "who are alice's co-workers?",
    },
    {
        "role": "assistant",
        "content": result.answer,
    }
]
graph_rag.search(
    "where is she living?",
    retriever_config={"top_k": 1},
    message_history=history,
)

RagResultModel(answer='Alice is living in Sweden.', retriever_result=None)